In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
from typing import Dict

In [2]:
import sys
sys.path.append("../")

In [10]:
import constants
from core.tools.metrics import *
from core.models.stat_models import *
from core.models.baseline_rnn import *
from core.tools.visualize import *
from core.tools.time_series import *
from core.tools.data_import import *
import core.models.baseline_lstm as baseline_lstm
import core.tools.rnn_prepare as rnn_prepare

In [4]:
# Avaiable datasets, loaded from constants.
pprint(constants.DATA_DIR)
choice = None
while choice is None or choice not in constants.DATA_DIR.keys():
    if choice is not None:
        print("Invalid data location received, try again...")
    choice = input("Select Dataset >>> ")
FILE_DIR = constants.DATA_DIR[choice]

{'0': '/Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv',
 '1': '/home/ec2-user/AnnEconForecast/data/UNRATE.csv',
 '2': '/home/ubuntu/AnnEconForecast/data/UNRATE.csv',
 '3': '/home/ec2-user/AnnEconForecast/data/DEXCAUS.csv'}
Select Dataset >>> 1


In [5]:
print(f"Dataset chosen: \n{FILE_DIR}")

Dataset chosen: 
/home/ec2-user/AnnEconForecast/data/UNRATE.csv


In [6]:
# Pre-processing Parameters
PERIODS = 1
ORDER = 1
LAGS = 12
TRAIN_RATIO = 0.8

In [7]:
# # Neural Network Parameters
# NUM_TIME_STEPS = LAGS
# # Number of series used to predict. (including concurrent)
# NUM_INPUTS = 1
# NUM_OUTPUTS = 1
# NUM_NEURONS = 64
# # Number of output series
# LEARNING_RATE = 0.01
# EPOCHS = 50
# # Training Settings
# REPORT_PERIODS = 1
# TENSORBOARD_DIR = "./tensorboard/test/1"

In [11]:
EXPERIMENT_NAME = "local-demo"
TENSORBOARD_DIR = f"../tensorboard/{EXPERIMENT_NAME}"
MODEL_PATH = f"/home/ec2-user/saved_models/{EXPERIMENT_NAME}"

In [19]:
# Main Training Parameters
parameters = {
    "epochs": 500,
    "num_time_steps": LAGS,
    "num_inputs": 1,
    "num_outputs": 1,
    "num_neurons": 128,
    "learning_rate": 0.1,
    "report_periods": 10,
    "tensorboard_dir": TENSORBOARD_DIR,
    "model_path": MODEL_PATH
}

In [20]:
prepared_df = rnn_prepare.prepare_dataset(
    file_dir=FILE_DIR,
    periods=PERIODS,
    order=ORDER
)

Retrieving raw data from /home/ec2-user/AnnEconForecast/data/UNRATE.csv...
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
Processing data, taking (periods, order)=(1, 1)...
Dropping Nan observations...
First few rows of dataset loaded:
            UNRATE_period1_order1
DATE                             
1948-02-01                    0.4
1948-03-01                    0.2
1948-04-01                   -0.1
1948-05-01                   -0.4
1948-06-01                    0.1


In [21]:
# Normalize data
(X_train, X_val, X_test,
 y_train, y_val, y_test) = rnn_prepare.generate_splited_dataset(
    raw=prepared_df,
    train_ratio=0.8,
    val_ratio=0.1,
    lags=LAGS
)

StandardScaler applied, scaling based on the first 679 observations.
Total 836 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (836, 12, 1), y shape = (836, 1, 1)
Training and testing set generated,        
X_train shape: (668, 12, 1)        
y_train shape: (668, 1)        
X_test shape: (84, 12, 1)        
y_test shape: (84, 1)        
X_validation shape: (84, 12, 1)        
y_validation shape: (84, 1)


In [25]:
data_collection = {
    "X_train": X_train,
    "X_val": X_val,
    "X_test": X_test,
    "y_train": y_train,
    "y_val": y_val,
    "y_test": y_test
}

In [26]:
checkpoints = lambda z: [z*x for x in range(1, parameters["epochs"]//z)] + [-1]

In [27]:
# Run the model
(metrics_dict, predictions) = baseline_lstm.exec_core(
    parameters=parameters,
    data_collection=data_collection,
    clip_grad=None,
    prediction_checkpoints=checkpoints(50)
)

Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.


ResourceExhaustedError: OOM when allocating tensor with shape[668,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node RECURRENT_UNITS/rnn/while/LSTM_Cell/MatMul (defined at ../core/models/baseline_lstm.py:86)  = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](RECURRENT_UNITS/rnn/while/LSTM_Cell/concat, RECURRENT_UNITS/rnn/while/LSTM_Cell/MatMul/Enter)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'RECURRENT_UNITS/rnn/while/LSTM_Cell/MatMul', defined at:
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/te

In [ ]:
# plt.close()
# plt.figure(figsize=(32, 16))
# plt.plot(predictions[1000]["train"].reshape(-1, 1), alpha=0.6)
# plt.plot(y_train.reshape(-1, 1), alpha=0.6)
# plt.legend(["Training Prediction", "Training Actual"])
# plt.grid(True)
# plt.title("Training Set Result")
# plt.show()

In [ ]:
fig = plot_checkpoints(predictions, y_test, "test")

In [17]:
# plt.close()
# plt.figure(figsize=(32, 16))
# plt.plot(predictions[100]["test"].reshape(-1, 1), alpha=0.6)
# plt.plot(y_test.reshape(-1, 1), alpha=0.6)
# plt.legend(["Testing Prediction", "Testing Actual"])
# plt.grid(True)
# plt.title("Testing Set Result")
# plt.show()